<a href="https://colab.research.google.com/github/Srajan1122/Hyperspectral_Image_Classification/blob/srajan/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Importing Libraries

In [2]:
import pandas as pd
import numpy as np
from scipy import io
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder 
from sklearn import preprocessing
import keras

Using TensorFlow backend.


# Importing Dataset

In [0]:
loaded_dataset = io.loadmat('/content/drive/My Drive/Srajan/Internship/SEM4-LeadingindiaAI/Tasks/Datasets/PaviaC/Pavia.mat')
for key, value in loaded_dataset.items():
  if isinstance(value, type(np.array([1]))):
    image = loaded_dataset[key]

In [0]:
ground_truth_1 = io.loadmat('/content/drive/My Drive/Srajan/Internship/SEM4-LeadingindiaAI/Tasks/Datasets/PaviaC/Pavia_gt.mat')
for key, value in ground_truth_1.items():
  if isinstance(value, type(np.array([1]))):
    ground_truth = ground_truth_1[key]

## Class Labels

In [0]:
label_values = ["Alfalfa", "Corn-notill", "Corn-mintill",
                        "Corn", "Grass-pasture", "Grass-trees",
                        "Grass-pasture-mowed", "Hay-windrowed", "Oats",
                        "Soybean-notill", "Soybean-mintill", "Soybean-clean",
                        "Wheat", "Woods", "Buildings-Grass-Trees-Drives",
                        "Stone-Steel-Towers"]

## Resizing the data

In [0]:
image_with_gt = np.dstack((image, ground_truth))

In [0]:
final_output = image_with_gt.reshape(ground_truth.size, image.shape[2]+1)

## Data Visualization in pandas

In [0]:
data = pd.DataFrame(final_output)

In [9]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102
0,854,601,350,266,138,118,178,194,257,269,239,243,220,191,173,150,125,131,137,172,195,218,255,291,320,346,347,368,404,428,429,416,394,368,357,361,369,368,351,328,...,219,239,289,366,482,632,814,1005,1225,1464,1732,2044,2383,2716,2985,3203,3385,3538,3667,3696,3548,3608,3744,3792,3802,3815,3836,3827,3765,3752,3759,3773,3779,3752,3690,3671,3664,3636,3643,0
1,527,642,575,294,123,168,207,154,209,299,299,278,235,181,181,236,256,253,225,202,221,256,321,358,385,429,492,533,554,553,557,580,561,522,525,513,474,435,412,401,...,254,305,349,401,508,681,888,1076,1297,1577,1855,2117,2456,2772,3002,3205,3388,3526,3629,3665,3586,3653,3747,3813,3831,3829,3841,3885,3916,3907,3873,3902,3921,3861,3854,3882,3834,3725,3768,0
2,374,322,179,87,169,268,360,339,286,309,340,305,206,202,269,256,246,226,218,275,318,356,395,425,461,520,587,623,621,617,636,657,665,670,645,610,576,547,532,523,...,369,418,484,590,754,969,1201,1460,1716,2002,2319,2690,3055,3368,3666,3901,4082,4258,4372,4392,4231,4256,4419,4495,4501,4472,4486,4499,4453,4404,4443,4472,4428,4353,4306,4284,4318,4311,4321,0
3,706,520,560,572,425,243,271,272,258,276,299,313,320,336,299,274,322,361,358,377,387,396,415,486,564,624,668,680,696,728,753,721,700,713,690,642,592,560,542,547,...,384,418,481,568,693,865,1057,1259,1478,1714,1971,2261,2548,2818,3135,3377,3564,3731,3820,3814,3710,3770,3904,3976,3967,3973,4032,4078,4040,3992,3972,4006,4032,3975,3946,3954,3944,3936,3939,0
4,1120,1027,592,414,407,463,417,365,332,334,356,327,296,325,346,308,262,262,292,312,335,394,430,450,491,570,611,616,631,659,686,705,691,667,624,584,570,549,530,517,...,349,365,398,505,677,877,1116,1410,1717,2015,2339,2670,3028,3415,3713,3928,4131,4296,4445,4508,4305,4232,4422,4554,4550,4524,4532,4578,4584,4555,4502,4485,4479,4445,4364,4290,4268,4235,4272,0


## Droping the rows if ground truth value is zero

In [10]:
print("Percentage of column which will be droped",(data.size - data[data.iloc[:, -1] == 0].size)/data.size,"%")

Percentage of column which will be droped 0.1890561992751774 %


In [0]:
data = data[data.iloc[:, -1] != 0]

# Spliting the data into feature and target

In [0]:
X = data.iloc[:, :-1]
y = data.iloc[:,-1]

# Feature Selection

In [0]:
# from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import chi2, f_classif, mutual_info_classif

# X = SelectKBest(f_classif, k=int((image.shape[2]+1)*0.75)).fit_transform(X, y)

# Feature Extraction 


In [0]:
from sklearn.decomposition import PCA 

pca = PCA(n_components = int((image.shape[2]+1)*0.75))
X = pca.fit_transform(X)

# OneHotEncoding in target column

In [0]:
onehotencoder = OneHotEncoder() 
y = onehotencoder.fit_transform(np.array(y).reshape(-1,1)).toarray() 

# Standardizing the feature columna

In [0]:
X = preprocessing.scale(X)

# Spliting the data into training and testing set 

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.90, random_state=1)#0.25 0.15

In [0]:
number = int(X_test.shape[0]/2)

xx_test = X_test[:number, :]
xx_val = X_test[number:, :]

if len(y_test.shape) > 1:
  loss = keras.losses.categorical_crossentropy
  metrics=['accuracy']
  yy_test = y_test[:number, :]
  yy_val = y_test[number:, :]
else:
  loss = keras.losses.sparse_categorical_crossentropy
  metrics=['sparse_categorical_accuracy']
  y_test = [x - 1 for x in y_test]
  y_train = [x - 1 for x in y_train]
  yy_test = y_test[:number]
  yy_val = y_test[number:]

In [0]:
from keras import backend as K
feature = X.shape[1]

if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, feature)
    xx_test = xx_test.reshape(xx_test.shape[0], 1, feature)
    xx_val = xx_val.reshape(xx_val.shape[0], 1, feature)
    input_shape = (1, feature)
else:
    X_train = X_train.reshape(X_train.shape[0], feature, 1)
    xx_test = xx_test.reshape(xx_test.shape[0], feature, 1)
    xx_val = xx_val.reshape(xx_val.shape[0], feature, 1)
    input_shape = (feature, 1)

# Genetic Algorithm

In [20]:
!pip install pyeasyga

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D, BatchNormalization

In [22]:
from pyeasyga import pyeasyga
import random
import numpy as np
from sklearn.metrics import accuracy_score
import keras
import time

count = 1

def initilialize_population():
    filter_1 = list(x for x in range(1, 11))
    filter_2 = list(x for x in range(11, 31))
    filter_3 = list(x for x in range(31, 61))
    filter_4 = list(x for x in range(61, 101))

    kernel_size_1 = [2,3]
    kernel_size_2 = [2,3]
    kernel_size_3 = [2,3]
    kernel_size_4 = [2,3]


    population = []
    population.append(filter_1)
    population.append(filter_2)
    population.append(filter_3)
    population.append(filter_4)
    population.append(kernel_size_1)
    population.append(kernel_size_2)
    population.append(kernel_size_3)
    population.append(kernel_size_4)

    return list(population)

def create_individual(data):
  choice = []
  for i in range(len(data)):
    choice.append(random.choice(list(range(len(data[i])))))
  return choice

def fitness_function(individual, data):
  global count
  print('Individual no', count)
  count += 1
  model = Sequential()
  model.add(Conv1D(filters=data[0][individual[0]], kernel_size=data[4][individual[4]], activation='relu', input_shape=input_shape))
  model.add(Conv1D(filters=data[0][individual[0]], kernel_size=data[4][individual[4]], activation='relu', input_shape=input_shape))
  model.add(BatchNormalization())
  model.add(MaxPooling1D(pool_size=2))
  model.add(Conv1D(filters=data[1][individual[1]], kernel_size=data[5][individual[5]], activation='relu'))
  model.add(Conv1D(filters=data[1][individual[1]], kernel_size=data[5][individual[5]], activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling1D(pool_size=2))
  model.add(Conv1D(filters=data[2][individual[2]], kernel_size=data[6][individual[6]], activation='relu'))
  model.add(Conv1D(filters=data[2][individual[2]], kernel_size=data[6][individual[6]], activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling1D(pool_size=2))
  model.add(Conv1D(filters=data[3][individual[3]], kernel_size=data[7][individual[7]], activation='relu'))
  model.add(Conv1D(filters=data[3][individual[3]], kernel_size=data[7][individual[7]], activation='relu'))
  model.add(BatchNormalization())
  model.add(MaxPooling1D(pool_size=2))
  model.add(Dropout(0.25))
  model.add(Flatten())
  model.add(Dense(64, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.25))
  model.add(Dense(32, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.25))
  model.add(Dense(16, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dropout(0.25))
  model.add(Dense(9, activation='softmax'))

  model.compile(loss=loss,
                optimizer=keras.optimizers.Adadelta(),
                metrics=metrics)

  model.fit(X_train, np.array(y_train),
            batch_size=128,
            epochs=25,
            verbose=1,
            validation_data=(xx_val,np.array(yy_val)))


  score = model.evaluate(xx_test, np.array(yy_test), verbose=1)[1]
  # prediction = model.predict(X_test)
  # score = accuracy_score(y_test, prediction)
  print('Score is', score)
  return score
  
data = initilialize_population()
ga = pyeasyga.GeneticAlgorithm(seed_data=data,
                               population_size=2,
                               generations=2,
                               crossover_probability=0.8,
                               mutation_probability=0.4,
                               elitism=True,
                               maximise_fitness=True)
ga.create_individual = create_individual
ga.fitness_function = fitness_function

start = time.time()
ga.run()
end = time.time()
print('Time taken ------>', end-start)

Individual no 1
Train on 14815 samples, validate on 66669 samples
Epoch 1/25
14815/14815 [==============================] - 12s 803us/step - loss: 1.7953 - accuracy: 0.4668 - val_loss: 1.9333 - val_accuracy: 0.4435
Epoch 2/25
14815/14815 [==============================] - 10s 694us/step - loss: 1.3182 - accuracy: 0.6279 - val_loss: 1.6217 - val_accuracy: 0.4688
Epoch 3/25
14815/14815 [==============================] - 10s 697us/step - loss: 1.1391 - accuracy: 0.6561 - val_loss: 1.1302 - val_accuracy: 0.6445
Epoch 4/25
14815/14815 [==============================] - 10s 691us/step - loss: 0.9430 - accuracy: 0.6994 - val_loss: 0.9712 - val_accuracy: 0.6790
Epoch 5/25
14815/14815 [==============================] - 10s 685us/step - loss: 0.7956 - accuracy: 0.7360 - val_loss: 0.6809 - val_accuracy: 0.7727
Epoch 6/25
14815/14815 [==============================] - 10s 679us/step - loss: 0.6975 - accuracy: 0.7667 - val_loss: 0.6296 - val_accuracy: 0.7971
Epoch 7/25
14815/14815 [================

In [23]:
best_parameters = ga.best_individual()[1]
for index,i in enumerate(data):
  print(i[best_parameters[index]])

print(ga.best_individual()[0])

7
30
44
88
2
3
2
3
0.9405561685562134


# Building CNN Model


In [24]:
model = Sequential()
model.add(Conv1D(filters=data[0][best_parameters[0]], kernel_size=data[4][best_parameters[4]], activation='relu', input_shape=input_shape))
model.add(Conv1D(filters=data[0][best_parameters[0]], kernel_size=data[4][best_parameters[4]], activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=data[1][best_parameters[1]], kernel_size=data[5][best_parameters[5]], activation='relu'))
model.add(Conv1D(filters=data[1][best_parameters[1]], kernel_size=data[5][best_parameters[5]], activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=data[2][best_parameters[2]], kernel_size=data[5][best_parameters[5]], activation='relu'))
model.add(Conv1D(filters=data[2][best_parameters[2]], kernel_size=data[5][best_parameters[5]], activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=data[3][best_parameters[3]], kernel_size=data[6][best_parameters[6]], activation='relu'))
model.add(Conv1D(filters=data[3][best_parameters[3]], kernel_size=data[6][best_parameters[6]], activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(16, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(9, activation='softmax'))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_33 (Conv1D)           (None, 76, 7)             21        
_________________________________________________________________
conv1d_34 (Conv1D)           (None, 75, 7)             105       
_________________________________________________________________
batch_normalization_29 (Batc (None, 75, 7)             28        
_________________________________________________________________
max_pooling1d_17 (MaxPooling (None, 37, 7)             0         
_________________________________________________________________
conv1d_35 (Conv1D)           (None, 35, 30)            660       
_________________________________________________________________
conv1d_36 (Conv1D)           (None, 33, 30)            2730      
_________________________________________________________________
batch_normalization_30 (Batc (None, 33, 30)           

In [25]:
for i in model.layers:
  name = i.output.name.split('/')[0].split('_')
  if len(name) > 2:
    name = name[0] + '_' + name[1]
  else:
    name = name[0]
  print(name, i.output.shape)
  print('_________________________________________________')

conv1d (None, 76, 7)
_________________________________________________
conv1d (None, 75, 7)
_________________________________________________
batch_normalization (None, 75, 7)
_________________________________________________
max_pooling1d (None, 37, 7)
_________________________________________________
conv1d (None, 35, 30)
_________________________________________________
conv1d (None, 33, 30)
_________________________________________________
batch_normalization (None, 33, 30)
_________________________________________________
max_pooling1d (None, 16, 30)
_________________________________________________
conv1d (None, 14, 44)
_________________________________________________
conv1d (None, 12, 44)
_________________________________________________
batch_normalization (None, 12, 44)
_________________________________________________
max_pooling1d (None, 6, 44)
_________________________________________________
conv1d (None, 5, 88)
_________________________________________________
conv1d (No

In [26]:
import keras
model.compile(loss=loss,
              optimizer=keras.optimizers.Adadelta(),
              metrics=metrics)

model.fit(X_train, np.array(y_train),
          batch_size=128,
          epochs=70,
          verbose=1,
          validation_data=(xx_val, np.array(yy_val)))

Train on 14815 samples, validate on 66669 samples
Epoch 1/70
14815/14815 [==============================] - 12s 809us/step - loss: 1.7803 - accuracy: 0.4580 - val_loss: 2.3484 - val_accuracy: 0.4434
Epoch 2/70
14815/14815 [==============================] - 10s 682us/step - loss: 1.3291 - accuracy: 0.6313 - val_loss: 2.7242 - val_accuracy: 0.4439
Epoch 3/70
14815/14815 [==============================] - 10s 685us/step - loss: 1.2011 - accuracy: 0.6518 - val_loss: 1.6243 - val_accuracy: 0.5473
Epoch 4/70
14815/14815 [==============================] - 10s 682us/step - loss: 1.1132 - accuracy: 0.6624 - val_loss: 1.1816 - val_accuracy: 0.6354
Epoch 5/70
14815/14815 [==============================] - 10s 684us/step - loss: 1.0085 - accuracy: 0.6827 - val_loss: 0.9731 - val_accuracy: 0.6729
Epoch 6/70
14815/14815 [==============================] - 10s 681us/step - loss: 0.8439 - accuracy: 0.7203 - val_loss: 0.7570 - val_accuracy: 0.7381
Epoch 7/70
14815/14815 [==============================] 

In [27]:
score = model.evaluate(xx_test, yy_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

66668/66668 [==============================] - 8s 127us/step
Test loss: 0.23187191087843054
Test accuracy: 0.9566508531570435


In [0]:
if len(np.array(y_test).shape) == 1:
  y_test = [x+1 for x in y_test]
  y_train = [x+1 for x in y_train]